# 1 context manager

# 2 Function

In [1]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [2]:
def reverse(word):
    return word[::-1]
reverse('hello')
sorted(fruits,key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## map()
特别是一个序列做map,直接用生成器表达式替换.

In [6]:
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

display(list(map(factorial, range(6))))
[factorial(n) for n in range(6) if n%2]

[1, 1, 2, 6, 24, 120]

[1, 6, 120]

## 函数内置属性

In [19]:
for n in dir(factorial):
    print(n,end='/')
print('\n')
fac = factorial
print(fac.__hash__())

__annotations__/__call__/__class__/__closure__/__code__/__defaults__/__delattr__/__dict__/__dir__/__doc__/__eq__/__format__/__ge__/__get__/__getattribute__/__globals__/__gt__/__hash__/__init__/__init_subclass__/__kwdefaults__/__le__/__lt__/__module__/__name__/__ne__/__new__/__qualname__/__reduce__/__reduce_ex__/__repr__/__setattr__/__sizeof__/__str__/__subclasshook__/

283942422


## 仅限关键字键值对的函数

In [45]:
def f(a,*,b=1):
    return a,b
f(1,b=9)

(1, 9)

In [49]:
fac = factorial
display(f.__code__)
display(f.__defaults__) # 为什么是None,仅定位参数?
display(fac.__defaults__)
f.__kwdefaults__

<code object f at 0x10f36d030, file "<ipython-input-45-bd6c10d9a2f7>", line 1>

None

None

{'b': 1}

In [48]:
from inspect import signature
sig = signature(f)
display(str(sig))
for name,p in sig.parameters.items():
     print(f'{name}={p.default}, {p.kind}')

'(a, *, b=1)'

a=<class 'inspect._empty'>, 1
b=1, 3


In [22]:
r = f(a=0,b=8)
r[0]

0

## 支持函数式编程的包
高阶函数演示.

In [50]:
from functools import reduce
from operator import mul

def fact_re(n):
    """ reduce 结合 mul 计算阶乘 """
    return reduce(mul,range(1,n+1))

fact_re(5)

120

### itemgetter()

In [2]:
from operator import itemgetter
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
cc_name = itemgetter(1,0)
for c in metro_data:
     print(cc_name(c))

itemgetter(3)(metro_data[3])

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


(40.808611, -74.020386)

### attrgetter() and namedtuple()

In [4]:
from operator import attrgetter
from collections import namedtuple

LatLong = namedtuple('LL','lat long')
Metro = namedtuple('Metro','name cc pop corrd')
metro_area = [Metro(name,cc,pop,LatLong(lat,long)) for name,cc,pop,(lat,long) in metro_data]

attrgetter('name','corrd.lat','corrd.long')(metro_area[2])

('Mexico City', 19.433333, -99.133333)

### functools.partial

In [7]:
from operator import mul
from functools import partial

triple = partial(mul,3)
display(triple(7))
display(list(map(triple,range(1,11))))

21

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]

# 3 Iterator and Generator
From Fluent Python Chapter 14.

## Iterable 和 Iterator

动态在pathlib中插入src

In [2]:
import sys
import os
from wcmatch import pathlib

display(sys.executable)
src_p = pathlib.Path("..") / "src"
sys.path.insert(0, os.fspath(src_p.resolve()))

'/Users/lhc/.pyenv/versions/data/bin/python3.8'

In [4]:
from fluentpy.c14.sentence import Sentence1

s = Sentence1("Crazy world, extraordinary world.")
display(s)
i = iter(s)
for word in i:
    print(word)

for word in s:
    print(word)

# i 中已无可迭代的内容.
next(i)

Sentence 'Crazy world,...dinary world.'

Crazy
world
extraordinary
world
Crazy
world
extraordinary
world


StopIteration: 

In [20]:
from fluentpy.c14.sentence import Sentence2

s = Sentence2("Crazy world, extraordinary world.")
for word in s:
    print(word)

# iter()每次构造新的迭代器
display(next(iter(s)))
display(next(iter(s)))

# 新的迭代器,重新开始
# while 不会捕捉stopiteration异常
i = iter(s)
while(True):
    display(next(i))



Crazy
world
extraordinary
world


'Crazy'

'Crazy'

'Crazy'

'world'

'extraordinary'

'world'

StopIteration: 

## 迭代器中使用生成器函数 yield

In [7]:
from fluentpy.c14.sentence import Sentence3

s = Sentence3("Sentence3, Crazy world, extraordinary world.")
for word in s:
    print(word)

next(iter(s))

in Sentence 3.
Sentence3
in Sentence 3.
Crazy
in Sentence 3.
world
in Sentence 3.
extraordinary
in Sentence 3.
world
in Sentence 3.


'Sentence3'

In [6]:
from fluentpy.c14.sentence import Sentence4

s = Sentence4("Sentence 4, Crazy world, extraordinary world.")
for word in s:
    print(word)


Sentence
4
Crazy
world
extraordinary
world
end
with
sentence
4


'Sentence'

## 生成器函数:yield from(广度和深度搜索)

In [5]:
from pycookbook.c4.node44 import Node

root = Node(0)
c1 = Node(1)
c2 = Node(2)
c3 = Node(3)

root.add_child(c1)
root.add_child(c2)
root.add_child(c3)

c1.add_child(Node(11))
c1.add_child(Node(12))
c2.add_child(Node(21))
c3.add_child(Node(31))

for ch in root.depth_first():
    print(f'out print:{ch}')

print('======================')
for ch in root.depth_first_yield_from():
    print(f'out print:{ch}')

print('======================')
isRoot = True
for ch in root.width_first(isRoot):
    print(f'out print:{ch}')
    isRoot = False

before yield:Node(0)
out print:Node(0)
after yield:Node(0)
in for Node(1)
before yield:Node(1)
Node(1)
after yield:Node(1)
in for Node(11)
before yield:Node(11)
Node(11)
after yield:Node(11)
in for Node(12)
before yield:Node(12)
Node(12)
after yield:Node(12)
in for Node(2)
before yield:Node(2)
Node(2)
after yield:Node(2)
in for Node(21)
before yield:Node(21)
Node(21)
after yield:Node(21)
in for Node(3)
before yield:Node(3)
Node(3)
after yield:Node(3)
in for Node(31)
before yield:Node(31)
Node(31)
after yield:Node(31)
inner print:before yield:Node(0)
out print:Node(0)
inner print:after yield:Node(0)
inner print:before yield:Node(1)
out print:Node(1)
inner print:after yield:Node(1)
inner print:before yield:Node(11)
out print:Node(11)
inner print:after yield:Node(11)
inner print:before yield:Node(12)
out print:Node(12)
inner print:after yield:Node(12)
inner print:before yield:Node(2)
out print:Node(2)
inner print:after yield:Node(2)
inner print:before yield:Node(21)
out print:Node(21)
inn